In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5" 
import logging
import torch
print("Visible devices:", torch.cuda.device_count())
print("Current device ID:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))
from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_error() 
import sys
sys.path.append(".")
from main import run_single_agent, run_multi_agent
from agents.intent_enhanced_retrieval import intent_enhanced_reformulation
from chroma.chroma import ChromaClient
chroma_store_path = os.path.abspath("./chroma/chroma_store/evidence_bgebase")

chroma_client = ChromaClient(vector_name="evidence_bgebase", path=chroma_store_path)

print("ChromaDB storage path:", chroma_store_path)

Visible devices: 1
Current device ID: 0
Device name: NVIDIA H100 80GB HBM3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ChromaDB storage path: /home/yirui/mad/chroma/chroma_store/evidence_bgebase


In [3]:
# === Simple Retrieval ===
claim = "Gas prices were WAY higher in 2008, under REPUBLICAN president George W. Bush."
results = chroma_client.query(query_text=claim, top_k=20, include=["documents", "metadatas"])
evidence_list = []
for i, (text, meta) in enumerate(zip(results["documents"][0], results["metadatas"][0])):
    evidence_list.append(text)
    print(f"TOP{i} : [{meta['evidence_id']}]\n{text}\n")
evidence_text = "\n".join(evidence_list)

TOP0 : [1]
"You won’t see THIS on Fox ‘News’: Gas prices were WAY higher in 2008, under REPUBLICAN president George W. Bush," Occupy Democrats wrote on Facebook. The post included images of two gas station price signs, labeled 2008 and 2021.

TOP1 : [2]
Statistics from the U.S. Energy Information Administration back up the claim about prices. During the past 20 years, the highest gas prices occurred in July 2008, during the Bush administration.

TOP2 : [7]
But Bush’s term also included two major economic shocks that sent gasoline prices tumbling. The first was the 9/11 attacks in 2001. By the end of that year, gas prices were around $1.07. The other was the 2008 mortgage crisis and Wall Street collapse, near the end of his second term. By the time Obama took over, gas prices were down to about $1.83.

TOP3 : [20227]
Gasoline prices are surely higher under Biden than they were under his predecessor, former President Donald Trump.

TOP4 : [23791]
Gasoline prices are higher under Biden th

In [3]:
# === Single-Agent Fact-Checking ===
result = run_single_agent(claim, evidence_text)
print(result)

VERDICT: HALF-TRUE
REASON: The claim states that gas prices were WAY higher in 2008 under Republican president George W. Bush. While it is true that gas prices were higher in 2008, the claim is exaggerated. According to the evidence, the highest gas prices occurred in July 2008, but the claim does not specify the exact prices. Additionally, the evidence shows that gas prices were affected by two major economic shocks during Bush's term, which led to a decrease in prices towards the end of his term. The claim also does not account for inflation, which would affect the comparison between prices in 2008 and 2021. Therefore, the claim is half-true because it is partially accurate but also misleading and incomplete.


In [4]:
# === Multi-Agent Fact-Checking ===
pro_open, con_open, pro_rebut, con_rebut, pro_close, con_close, final_result = run_multi_agent(claim, evidence_text)

print(f"pro_opening:\n{pro_open}\n")
print(f"con_opening:\n{con_open}\n")
print(f"pro_rebuttal:\n{pro_rebut}\n")
print(f"con_rebuttal:\n{con_rebut}\n")
print(f"pro_closing:\n{pro_close}\n")
print(f"con_closing:\n{con_close}\n")
print(f"final_verdict:\n{final_result}\n")


=== Running Multi-Agent Debate (3 rounds) ===
pro_opening:
Good evening, esteemed debaters. I'm here to present my opening argument in support of the claim that gas prices were indeed higher in 2008, under the presidency of George W. Bush.

Firstly, let's look at the evidence provided by the U.S. Energy Information Administration (EIA). According to the EIA, the highest gas prices in the past 20 years occurred in July 2008, during the Bush administration. This is a verifiable fact that cannot be disputed.

Furthermore, the EIA statistics show that gas prices were significantly higher in 2008 compared to other years. In fact, the average gas price in July 2008 was around $4.11 per gallon, which is much higher than the current average price of around $3.32 per gallon under the Biden administration.

It's also important to note that the 2008 gas prices were not solely due to the Bush administration's policies. The global financial crisis and the subsequent economic downturn also played a

In [9]:
# === Intent Enhanced Retrieval ===
# Step 1: Infer Intent
result = intent_enhanced_reformulation(claim)
pro_claim = result["reformulated_pro"]
con_claim = result["reformulated_con"]
print(f"pro_claim:\n{pro_claim}\n")
print(f"con_claim:\n{con_claim}\n")
intent_evidence_list = []
intent_evidence_pro = []
intent_evidence_con = []


# Step 2: Query ChromaDB using pro_claim
pro_results = chroma_client.query(query_text=pro_claim, top_k=10, include=["documents", "metadatas"])
for i, (text, meta) in enumerate(zip(pro_results["documents"][0], pro_results["metadatas"][0])):
    intent_evidence_list.append(text)
    intent_evidence_pro.append(text)
    print(f"PRO_TOP{i} : [{w['evidence_id']}]\n{text}\n")


# Step 3: Query ChromaDB using con_claim
con_results = chroma_client.query(query_text=con_claim, top_k=10, include=["documents", "metadatas"])
for i, (text, meta) in enumerate(zip(con_results["documents"][0], con_results["metadatas"][0])):
    intent_evidence_list.append(text)
    intent_evidence_con.append(text)
    print(f"CON_TOP{i} : [{meta['evidence_id']}]\n{text}\n")

intent_evidence_text = "\n".join(intent_evidence_list)
intent_evidence_pro_text = "\n".join(intent_evidence_pro)
intent_evidence_con_text = "\n".join(intent_evidence_con)

pro_claim:
Gas prices were significantly higher under Republican President George W. Bush in 2008, indicating that the current administration's efforts to stabilize and reduce gas prices have been more effective than those of the previous administration.

con_claim:
"Gas prices varied significantly during 2008, with multiple factors influencing their fluctuations, including global economic conditions, global demand, and supply chain disruptions, making it difficult to attribute price changes solely to the presidency of George W. Bush, a Republican."

PRO_TOP0 : [1]
"You won’t see THIS on Fox ‘News’: Gas prices were WAY higher in 2008, under REPUBLICAN president George W. Bush," Occupy Democrats wrote on Facebook. The post included images of two gas station price signs, labeled 2008 and 2021.

PRO_TOP1 : [2]
Statistics from the U.S. Energy Information Administration back up the claim about prices. During the past 20 years, the highest gas prices occurred in July 2008, during the Bush ad

In [10]:
# === Intent Enhanced Single-Agent Fact-Checking with conbined evidence===
result = run_single_agent(claim, intent_evidence_text)
print(result)

VERDICT: HALF-TRUE
REASON: The claim that "Gas prices were WAY higher in 2008, under REPUBLICAN president George W. Bush" is partially true. According to the evidence, gas prices did reach a peak of around $4.11 in July 2008 during the Bush administration. However, the claim is also misleading because it does not provide the full context of the gas price fluctuations during that time. The evidence also shows that gas prices were affected by major economic shocks, such as the 9/11 attacks and the 2008 mortgage crisis, which led to a decline in gas prices by the time Obama took office. Additionally, the claim does not account for the fact that gas prices have fluctuated within and across administrations, with little correlation to which president or party is in power. The price of gasoline is determined by the state of the economy and the balance of supply and demand, rather than by presidential policies. Therefore, the claim is half-true because it is partially accurate but also mislead

In [11]:
# === Intent Enhanced Single-Agent Fact-Checking with seperate evidence===
evidence = f"### Pro-side Evidence:\n{intent_evidence_pro_text}\n\n### Con-side Evidence:\n{intent_evidence_con_text}"
result = run_single_agent(claim, intent_evidence_text)
print(result)

VERDICT: HALF-TRUE
REASON: The claim that "Gas prices were WAY higher in 2008, under REPUBLICAN president George W. Bush" is partially true. According to the evidence, gas prices did reach a peak of around $4.11 in July 2008 during the Bush administration. However, the claim is also misleading because it does not provide the full context of the gas price fluctuations during that time. The evidence also shows that gas prices were affected by major economic shocks, such as the 9/11 attacks and the 2008 mortgage crisis, which led to a decline in gas prices by the time Obama took office. Additionally, the claim does not account for the fact that gas prices have fluctuated within and across administrations, with little correlation to which president or party is in power. The price of gasoline is determined by the state of the economy and the balance of supply and demand, rather than by presidential policies. Therefore, the claim is half-true because it is partially accurate but also mislead

In [6]:
# === Intent Enhanced Multi-Agent Fact-Checking with conbined evidence===
pro_open, con_open, pro_rebut, con_rebut, pro_close, con_close, final_result = run_multi_agent(claim, intent_evidence_text)

print(f"pro_opening:\n{pro_open}\n")
print(f"con_opening:\n{con_open}\n")
print(f"pro_rebuttal:\n{pro_rebut}\n")
print(f"con_rebuttal:\n{con_rebut}\n")
print(f"pro_closing:\n{pro_close}\n")
print(f"con_closing:\n{con_close}\n")
print(f"final_verdict:\n{final_result}\n")


=== Running Multi-Agent Debate (3 rounds) ===
pro_opening:
I'm here to argue that the claim "Gas prices were WAY higher in 2008, under REPUBLICAN president George W. Bush" is TRUE. 

Firstly, the evidence from the U.S. Energy Information Administration (EIA) shows that the highest gas prices in the past 20 years occurred in July 2008, during the Bush administration. This is a fact that cannot be disputed.

Secondly, the graph provided in the evidence shows that gas prices have fluctuated over time, with a peak in 2008 and a decline in 2009. This graph is a visual representation of the data provided by the EIA, which is a reliable source of energy information.

Thirdly, the article mentions that the 9/11 attacks in 2001 and the 2008 mortgage crisis and Wall Street collapse had a significant impact on gas prices. This is true, as these events caused a decline in gas prices due to reduced demand. However, this does not change the fact that gas prices were higher in 2008 compared to other

In [12]:
# === Intent Enhanced Multi-Agent Fact-Checking with seperate evidence===
evidence = f"### Pro-side Evidence:\n{intent_evidence_pro_text}\n\n### Con-side Evidence:\n{intent_evidence_con_text}"
pro_open, con_open, pro_rebut, con_rebut, pro_close, con_close, final_result = run_multi_agent(claim, evidence)

print(f"pro_opening:\n{pro_open}\n")
print(f"con_opening:\n{con_open}\n")
print(f"pro_rebuttal:\n{pro_rebut}\n")
print(f"con_rebuttal:\n{con_rebut}\n")
print(f"pro_closing:\n{pro_close}\n")
print(f"con_closing:\n{con_close}\n")
print(f"final_verdict:\n{final_result}\n")


=== Running Multi-Agent Debate (3 rounds) ===
pro_opening:
I'm here to present the opening argument in support of the claim that gas prices were WAY higher in 2008, under REPUBLICAN president George W. Bush.

Firstly, let's look at the evidence provided by the U.S. Energy Information Administration (EIA). According to the EIA, the highest gas prices occurred in July 2008, during the Bush administration. This is a fact that cannot be disputed.

Secondly, the graph provided shows that gas prices have fluctuated over the years, but the peak price of around $4.11 in July 2008 is significantly higher than the current price of around $3.32 per gallon.

Thirdly, it's important to note that the 2008 mortgage crisis and Wall Street collapse had a significant impact on the economy, which in turn affected gas prices. However, this does not change the fact that gas prices were higher during the Bush administration.

In conclusion, the evidence clearly shows that gas prices were indeed WAY higher 